# VacationPy
----

In [92]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [93]:
cities_df = pd.read_csv("output_data/cities.csv")
del cities_df['Unnamed: 0'] #delete extra column with no named heading saved to csv file
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,butaritari,3.0707,172.7902,81.28,80,96,19.91,KI,1612730964
1,seoul,37.5683,126.9778,24.80,54,0,10.36,KR,1612730803
2,yetkul,54.8221,61.5880,-0.40,65,73,11.18,RU,1612730964
3,rikitea,-23.1203,-134.9692,77.65,72,96,11.97,PF,1612730964
4,pedernales,18.0384,-71.7440,84.69,52,33,3.29,DO,1612730965


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [94]:
gmaps.configure(api_key=g_key) #configures gmaps
locations=cities_df[["Lat","Lng"]]
humidity=cities_df["Humidity"]

#fig = gmaps.figure()
fig = gmaps.figure(center = (30,0), zoom_level=1.6) #used this center and zoom_level pair paramaters to zoom in on one panel (vs. default 3 panels) of the world map

#used code from 10-Stu_BankDeserts_Heatmap activity
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3) #max intensity set to the max humidity in the dataset, 100%
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [95]:
#according to HW instructions, the ideal weather should include:
  # 70*F < Max temp < 80*F
  # Wind speed < 10 mph
  # cloudiness = 0

#therefore, we will drop any cities that do not meet all of these conditions:
clouds = cities_df['Cloudiness']
temp = cities_df['Max Temp']
wind = cities_df['Wind Speed']
hotel_df = cities_df.loc[(clouds == 0) & (temp > 70) & (temp < 80) & (wind < 10)]
hotel_df.dropna()
hotel_df.head(10) #there are only 10 cities in the cities_df that meet all these criteria

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,cidreira,-30.1811,-50.2056,74.41,65,0,9.10,BR,1612730835
43,castro,-24.7911,-50.0119,70.77,66,0,4.59,BR,1612730968
46,yulara,-25.2406,130.9889,73.40,35,0,6.91,AU,1612730968
49,caborca,30.5838,-111.1010,75.00,22,0,5.99,MX,1612730968
96,guerrero negro,27.9769,-114.0611,77.02,18,0,7.40,MX,1612730769
122,pisco,-13.7000,-76.2167,75.20,78,0,4.61,PE,1612730974
171,coihaique,-45.5752,-72.0662,78.80,25,0,5.75,CL,1612730978
371,goure,13.9835,10.2704,72.00,14,0,3.00,NE,1612730995
411,aden,12.7794,45.0367,75.52,66,0,4.90,YE,1612730861
532,diffa,13.3154,12.6113,70.20,20,0,7.20,NE,1612731011


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [96]:
hotel_df["Hotel Name"] = "" #create an empty column for Hotel Name
hotel_df["Hotel Lat"]=""
hotel_df["Hotel Lng"]=""
hotel_df.head(10)

C:\Users\kqo2\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\kqo2\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\kqo2\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Lat,Hotel Lng
8,cidreira,-30.1811,-50.2056,74.41,65,0,9.10,BR,1612730835,,,
43,castro,-24.7911,-50.0119,70.77,66,0,4.59,BR,1612730968,,,
46,yulara,-25.2406,130.9889,73.40,35,0,6.91,AU,1612730968,,,
49,caborca,30.5838,-111.1010,75.00,22,0,5.99,MX,1612730968,,,
96,guerrero negro,27.9769,-114.0611,77.02,18,0,7.40,MX,1612730769,,,
122,pisco,-13.7000,-76.2167,75.20,78,0,4.61,PE,1612730974,,,
171,coihaique,-45.5752,-72.0662,78.80,25,0,5.75,CL,1612730978,,,
371,goure,13.9835,10.2704,72.00,14,0,3.00,NE,1612730995,,,
411,aden,12.7794,45.0367,75.52,66,0,4.90,YE,1612730861,,,
532,diffa,13.3154,12.6113,70.20,20,0,7.20,NE,1612731011,,,


In [97]:
from pprint import pprint

# find the closest hotel of each type of coordinates in hotel_df, using example from 04-Ins_NearestRestraunt and 07-Stu_Airport_Map activity
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "types": "hotel",
    "radius": 5000, #According to HW instructions, want to find the first hotel for each city located within 5000 meters of city's coordinates.
    "keyword": "hotel",
    "key": g_key,
}

for index, row in hotel_df.iterrows(): # use iterrows to iterate through hotel_df
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"  # change location each iteration while leaving original params in place
    response = requests.get(base_url, params=params).json() # assemble url and make API request
    #pprint(response)
    
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        hotel_df.loc[index, "Hotel Lat"] = response["results"][0]["geometry"]["location"]["lat"]
        hotel_df.loc[index, "Hotel Lng"] = response["results"][0]["geometry"]["location"]["lng"]
    except (IndexError): #returns blank values for index errors
        hotel_df.loc[index, "Hotel Name"] = ""
        hotel_df.loc[index, "Hotel Lat"] = ""
        hotel_df.loc[index, "Hotel Lng"] = ""
        pass #allows for loop to continue after try/except code

hotel_df=hotel_df.loc[hotel_df["Hotel Name"] != ""] #this code drops any cities without hotels
hotel_df.head(10)

C:\Users\kqo2\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Lat,Hotel Lng
8,cidreira,-30.1811,-50.2056,74.41,65,0,9.10,BR,1612730835,Hotel Castelo,-30.1779,-50.2035
43,castro,-24.7911,-50.0119,70.77,66,0,4.59,BR,1612730968,Central Palace Hotel - Castro (PR),-24.7913,-50.0078
46,yulara,-25.2406,130.9889,73.40,35,0,6.91,AU,1612730968,The Lost Camel Hotel - Ayers Rock Resort,-25.2407,130.983
49,caborca,30.5838,-111.1010,75.00,22,0,5.99,MX,1612730968,Hotel La Villa,30.5393,-111.114
96,guerrero negro,27.9769,-114.0611,77.02,18,0,7.40,MX,1612730769,Malarrimo Hotel and Restaurant,27.9681,-114.03
122,pisco,-13.7000,-76.2167,75.20,78,0,4.61,PE,1612730974,HOSTAL MIRAMAR,-13.7274,-76.2223
171,coihaique,-45.5752,-72.0662,78.80,25,0,5.75,CL,1612730978,Entre Cumbres Apart Hotel,-45.5725,-72.0708
411,aden,12.7794,45.0367,75.52,66,0,4.90,YE,1612730861,Sheraton Gold Mohur Hotel,12.769,44.9904
532,diffa,13.3154,12.6113,70.20,20,0,7.20,NE,1612731011,Complexe Hôtelier Du 18 Décembre,13.3127,12.5904


In [98]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Hotel Lat", "Hotel Lng"]]

In [99]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info) #need to the locations and hotel_info from cell just before this
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))